Unit stats are stored in a dictionary. Stats we care about for combat sim are HP(int), DPS(float), Range(bool), Armor(int), Attributes(set string), Targetable(set string), Bonuses(set string), Bonus DPS(float). We also want to track Minerals(int), Gas(int), Supply(int), and Time(int) for the linear program.

All stats assume no upgrades. Shields are added to health to get HP. Bonuses is a set of string attributes that this unit will deal bonus damage to. Bonus DPS is the damage the unit will deal to an enemy unit after accounting for bonuses. Targetable is a set of string attributes that this unit can, either ground and/or air. We are coding armor is a multiplicitive bonus to a units health.

In [1]:
import random
from sim_units import get_Units


Units = get_Units()

Four Terran units have been coded in this notebook to test the functionality of our code. While the rest of the units in the game could be added the same way, it is recommended that we instead save all of our unit stats in a csv file and import it as a database using pandas. If this is done, then the only thing we need to change is how our Unit class initializes itself.

In [2]:
#Terran_Units = {}
#Units = {}

Marine = {}
# Combat stats
Marine['hp'] = 45
Marine['dps'] = 9.8
Marine['ranged'] = True
Marine['armor'] = 0
Marine['attributes'] = {'Biological', 'Light', 'Ground'}
Marine['type'] = {'Ground'}
Marine['targetable'] = {'Ground', 'Air'}
Marine['bonuses'] = {}
Marine['bonus_dps'] = 0
Marine['healer'] = False
# Resource stats
Marine['mineral'] = 50
Marine['gas'] = 0
Marine['time'] = 18
Marine['supply'] = 1
Marine['race'] = 'Terran'

#Terran_Units['Marine'] = Marine
#Units['Marine'] = Marine

SCV = {}
# Combat stats
SCV['hp'] = 45
SCV['dps'] = 4.67
SCV['ranged'] = False
SCV['armor'] = 0
SCV['attributes'] = {'Biological', 'Light', 'Mechanical', 'Ground'}
SCV['targetable'] = {'Ground'}
SCV['type'] = {'Ground'}
SCV['bonuses'] = {}
SCV['bonus_dps'] = 0
SCV['healer'] = True
# Resource stats
SCV['mineral'] = 75
SCV['gas'] = 0
SCV['time'] = 12
SCV['supply'] = 1
SCV['race'] = 'Terran'
#Terran_Units['SCV'] = SCV
#Units['SCV'] = SCV

Medivac = {}
# Combat stats
Medivac['hp'] = 150
Medivac['dps'] = 0
Medivac['ranged'] = False
Medivac['armor'] = 1
Medivac['attributes'] = {'Armored', 'Mechanical', 'Air'}
Medivac['targetable'] = {}
Medivac['type'] = {'Air'}
Medivac['bonuses'] = {}
Medivac['bonus_dps'] = 0
Medivac['healer'] = True
# Resource stats
Medivac['mineral'] = 100
Medivac['gas'] = 100
Medivac['time'] = 30
Medivac['supply'] = 2
Medivac['race'] = 'Terran'
#Terran_Units['Medivac'] = Medivac
#Units['Medivac'] = Medivac

Hellion = {}
# Combat stats
Hellion['hp'] = 90
Hellion['dps'] = 4.48*5
Hellion['ranged'] = True
Hellion['armor'] = 0
Hellion['attributes'] = {'Light', 'Mechanical', 'Ground'}
Hellion['targetable'] = {'Ground'}
Hellion['type'] = {'Ground'}
Hellion['bonuses'] = {'Light'}
Hellion['bonus_dps'] = 3.4*5
Hellion['healer'] = False
# Resource stats
Hellion['mineral'] = 100
Hellion['gas'] = 0
Hellion['time'] = 21
Hellion['supply'] = 2
Hellion['race'] = 'Terran'
#Terran_Units['Hellion'] = Hellion
#Units['Hellion'] = Hellion

The SCV and Medivac names will be flagged in the simulator so that they repair/heal with priority. Medivacs heal a flat 12.6 hps whereas SCVs heal rate differs per unit.

There are other units that can restore hp in the game. Many Protoss units have shields that regenerate when not in combat, and all Zerg units restore health when not in combat. Since these effects only happen when out of combat, we can safely ignore them for our simulation. The Terran MULE unit has the same repair ability as the SCV, but the MULE is used almost exclusively to gather resources. There is one other unit that can restore health in combat, the Zerg Queen. However, the main function of the Queen is to produce other Zerg units with a secondary usage to spreed Creep Tumors and so the Queen's Transfusion healing ability is almost never used. Additionally, the energy cost of the Queen's Transfusion ability is quite high so players typically use it to heal buildings, not units.

SCV repair rates on that forum post are wrong. We can calculate the rate at which a mechanical unit is repaired by an SCV with
$$\frac{unit.max\_hp}{unit.time}$$
so the units health divided by the amount of time it takes to build that unit. Basically it takes the same amount of time to build a unit as it does to repair it from 1 hp, just a quarter of the cost to repair. To include repair costs, we will be increasing the cost of the SCV by 50\% instead of somehow including this combat simulation into our objective function to reduce costs

In [3]:
def get_healable_units(allies, attribute):
    """
    Helper function to be used with healing_units
    allies is list of friendly Units, attribute is a string
    either "Biological" or "Mechanical" that determines the
    type of unit that can be healed
    Randomly chooses an allied Unit that can be healed
    returns that chosen Unit, returns None if no unit can be healed
    """
    heal_targets = []
    for ally in allies:
        if (attribute in ally.attributes) and (ally.hp < ally.max_hp):
            heal_targets.append(ally)
    if len(heal_targets) == 0:
        return None
    else:
        index = random.randint(0,len(heal_targets)-1)
        target = heal_targets[index]
        return target

In [4]:
def healing_units(army):
    """
    Input is a list of Units in an army
    If applicable, Units with the "healer" tag restore
    health to friendly Units in their army.
    Medivacs are allowed to split their healing, but we are
    restricting SCVs to only heal one target due to the
    variablity of the SCV repair ability
    """
    for unit in army:
        if unit.healer:
            allies = army
            allies.remove(unit)
            if unit.name == 'Medivac':
                heal = 12.6
                attribute = 'Biological'
                # Medivacs split their hps the same way other units can split dps
                while ((heal > 0) and (get_healable_units(allies, attribute) is not None)):
                    target = get_healable_units(allies, attribute)
                    restore = target.max_hp - target.hp
                    temp_heal = heal
                    heal -= restore
                    restore -= temp_heal
                    target.hp = target.max_hp - restore
                    # prevent overhealing
                    if target.hp > target.max_hp:
                        target.hp = target.max_hp
                    unit.repaired = True
            elif unit.name == 'SCV':
                # SCVs will only be allowed to repair a single target
                attribute = 'Mechanical'
                if get_healable_units(allies, attribute) is not None:
                    target = get_healable_units(allies, attribute)
                    heal = target.max_hp / target.time
                    target.hp += heal
                    # prevent overhealing
                    if target.hp > target.max_hp:
                        target.hp = target.max_hp
                    unit.repaired = True

Function for running combat sim.
Input: dictionary of enemy army, dictionary of testing army
    dictionary of armies has unit name as keys, unit count as values
Output: dictionary of testing army

In [5]:
# test enemy army
enemy_army_comp = {}
enemy_army_comp['SCV'] = 10
enemy_army_comp['Marine'] = 10
enemy_army_comp['Medivac'] = 5

# test test_army
test_army_comp = {}
test_army_comp['SCV'] = 10
test_army_comp['Marine'] = 10
test_army_comp['Medivac'] = 5

We will keep track of individual units within our army simulation by using a custom army Unit class

In [6]:
class Unit:
    def __init__(self, name, army):
        """
        Creates a new Unit object with the stats of the given name
        """
        self.name = name
        if Units[name]['armor'] > 0:
            self.hp = Units[name]['hp'] * Units[name]['armor'] * 1.5
        else:
            self.hp = Units[name]['hp']
        self.max_hp = self.hp
        self.dps = Units[name]['dps']
        self.ranged = Units[name]['ranged']
        self.attributes = Units[name]['attributes']
        self.type = Units[name]['type']
        self.targetable = Units[name]['targetable']
        self.bonuses = Units[name]['bonuses']
        self.bonus_dps = Units[name]['bonus_dps']
        self.time = Units[name]['time']
        self.healer = Units[name]['healer']
        # self.repaired is used only for healing units so that they cannot
        # repair and attack in the same round
        # repaired = True if Unit has repaired that round
        self.repaired = False
        # children are used only for the Carrier/Interceptor interaction
        if name == 'Carrier':
            self.child = {}
            self.child[1] = Unit('Interceptor', army)
            self.child[2] = Unit('Interceptor', army)
            self.child[3] = Unit('Interceptor', army)
            self.child[4] = Unit('Interceptor', army)
            self.child[5] = None
            self.child[6] = None
            self.child[7] = None
            self.child[8] = None
            army.append(self.child[1])
            army.append(self.child[2])
            army.append(self.child[3])
            army.append(self.child[4])
            # child_time is used to keep track of time until able to
            # build a new Interceptor child
            self.child_time = 0
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return str(self.name) + "_HP:" + str(self.hp)

SyntaxError: invalid syntax (Temp/ipykernel_23764/2861938885.py, line 27)

In [ ]:
def get_health(army):
    """
    Input is list of Units in an army
    Returns the sum of the remaining units in that army
    """
    health = 0
    for unit in army:
        health += unit.hp
    return health

In [ ]:
def build_army(army_comp):
    """
    Input is a dictionary representing an army composition
    Returns a list of Units matching the army composition
    """
    army = []
    for name in army_comp:
        count = army_comp[name]
        while count >= 1:
            army.append(Unit(name, army))
            count -= 1
    return army

In [ ]:
def get_attackable_unit(unit, enemy_army):
    """
    Input is attacking Unit and list of Units in enemy army
    Returns a random Unit in enemy army that attacking unit can attack
    If no enemy Unit can be attacked, return None
    """
    # create list of enemies unit can attack
    # targeting type (ground/air) and if enemy has hp>0
    enemies = []
    for enemy in enemy_army:
        if enemy.hp > 0:
            targetable = 0
            for target_type in unit.targetable:
                for enemy_type in enemy.type:
                    targetable += int(enemy_type == target_type)
            if targetable > 0:
                enemies.append(enemy)
    if len(enemies) == 0:
        return None
    else:
        # randomly chooses an enemy to attack
        index = random.randint(0,len(enemies)-1)
        enemy = enemies[index]
        return enemy

In [ ]:
def deal_damage(army, enemy_army):
    """
    Input army is a list of attacking Units,
    enemy_army is list of Units being attacked.
    Calculates the damage dealt to enemy_army by all attacking units
    Updates the list of enemy Units with damaged health numbers
    """
    for unit in army:
        deal_unit_dps(unit, enemy_army)

In [ ]:
def deal_unit_dps(unit, enemy_army):
    """
    Input is attacking Unit and list of Units being attacked
    Calculates the damage dealt to enemy army by the single unit
    Updates the list of enemy Units with damaged health numbers
    """
    if not unit.repaired:
        damage = unit.dps
        bonus_dmg = unit.bonus_dps
        while (damage > 0) and (get_attackable_unit(unit, enemy_army) is not None):
            enemy = get_attackable_unit(unit, enemy_army)
            # check for bonus damage. bonus damage is kept seperate from
            # normal damage
            bonus = 0
            # so long as there is some bonus dps to damage, check if
            # there is at least one matching attribute
            if bonus_dmg > 0:
                for bonus_att in unit.bonuses:
                    for att in enemy.attributes:
                        bonus += int(bonus_att == att)
            if bonus > 0:
                bonus_dmg_tmp = bonus_dmg
                bonus_dmg -= enemy.hp
                enemy.hp -= bonus_dmg_tmp
            dmg_temp = damage
            damage -= enemy.hp
            enemy.hp -= dmg_temp
    # reset repaired status for next round
    unit.repaired = False

In [ ]:
def deal_ranged_dps(army, enemy_army):
    """
    Input army is a list of attacking Units,
    enemy_army is list of Units being attacked.
    Calculates the damage dealt to enemy_army only by ranged units
    Updates the list of enemy Units with damaged health numbers
    """
    for unit in army:
        if unit.ranged:
            deal_unit_dps(unit, enemy_army)

In [ ]:
def remove_dead_units(army):
    """
    Input is a list of Units in an army
    Removes all Units with hp <=0 from that list
    Returns updated list
    """
    new_army = []
    # removes normal dead units
    for unit in army:
        if unit.hp >= 0:
            new_army.append(unit)
        # if Interceptor dies, free up that child space for the Carrier
        if unit.name == 'Carrier':
            for n in unit.child:
                if unit.child[n] is not None:
                    if unit.child[n].hp <= 0:
                        unit.child[n] = None
    # removes alive Interceptors if parent Carrier is dead
    for unit in army:
        if (unit.name == 'Carrier') and (unit.hp <= 0):
            for n in unit.child:
                if unit.child[n] in new_army:
                    new_army.remove(unit.child[n])
    return new_army

This method is used to have Carriers build new Interceptors, if applicable

In [ ]:
def build_Interceptor(army):
    """
    Input is an army.
    For every Carrier in that army, if an Interceptor
    slot is availible, build a new Interceptor and
    add that Interceptor back into the army
    """
    for unit in army:
        if unit.name == 'Carrier':
            for n in unit.child:
                if unit.child[n] is None:
                    if unit.child_time == 0:
                        unit.child[n] = Unit('Interceptor', army)
                        army.append(unit.child[n])
                        unit.child_time = 9

The combat_sim function is the only function we should be calling in the rest of our code, the other functions are only helper functions.

In [ ]:
def combat_sim(army_comp1, army_comp2):
    """
    Input two army compositions written as dictionary with unit names as
    keys, unit counts as values. Simulates combat with the two army.
    The first army is assumed to be the enemy army, the second army is
    the army composition we are testing viability for.
    If combat takes longer than 10 minutes then it is assumed to be a
    stalemate and army2 is forcibly killed off
    Returns remaining army composition of army2
    """
    MAX_ROUNDS = 600
    
    army1 = build_army(army_comp1)
    army2 = build_army(army_comp2)
    round1 = True
    rounds = 0
    while ((get_health(army1) > 0) and (get_health(army2) > 0)
          and (rounds <= MAX_ROUNDS)):
        if round1:
            deal_ranged_dps(army1, army2)
            deal_ranged_dps(army2, army1)
            round1 = False
        else:
            healing_units(army1)
            healing_units(army2)
            deal_damage(army1, army2)
            deal_damage(army2, army1)
        army1 = remove_dead_units(army1)
        army2 = remove_dead_units(army2)
        rounds += 1
    
    if rounds >= MAX_ROUNDS:
        army2.clear()
    
    return army2

In [ ]:
combat_sim(enemy_army_comp, test_army_comp)

In [ ]:
# test enemy army
enemy_army_comp = {}
enemy_army_comp['SCV'] = 0
enemy_army_comp['Marine'] = 4
enemy_army_comp['Medivac'] = 0
enemy_army_comp['Hellion'] = 0

# test test_army
test_army_comp = {}
test_army_comp['SCV'] = 0
test_army_comp['Marine'] = 0
test_army_comp['Medivac'] = 0
test_army_comp['Hellion'] = 0
test_army_comp['Carrier'] = 3